<a href="https://colab.research.google.com/github/Vignesh-397/Image_Captioning/blob/main/Microsfot_GIT/Microsfot_GIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
INPUT_DIR = '/content/drive/MyDrive/Major Project 2024/Dataset/Custom_Dataset'
OUTPUT_DIR = '/content/drive/MyDrive/Major Project 2024/Dataset/Custom_Dataset/v1_outputs'

In [3]:
!pip install -q datasets
!pip install evaluate
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

In [4]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from PIL import Image

# Define dataset paths
image_folder = os.path.join(INPUT_DIR, 'Images')  # Change to your actual images folder path
caption_file = os.path.join(INPUT_DIR, 'captions.txt')# Change to your actual caption file




In [5]:
# Step 1: Read Captions File
data = []
with open(caption_file, "r", encoding="ISO-8859-1") as f:
    for line in f:
        parts = line.strip().split(",", 1)  # Split at the FIRST comma
        if len(parts) == 2:
            image_id, caption = parts
            data.append([image_id.strip(), caption.strip()])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["image", "caption"])

# Step 2: Create Image-Caption Mapping
image_caption_dict = {}
for _, row in df.iterrows():
    image_name, caption = row["image"], row["caption"]
    if image_name in image_caption_dict:
        image_caption_dict[image_name].append(caption)
    else:
        image_caption_dict[image_name] = [caption]

# Step 3: Convert to Hugging Face Dataset Format (Lazy Image Loading)
dataset_list = []
for image_name, captions in image_caption_dict.items():
    image_path = os.path.join(image_folder, image_name + ".jpg")  # Ensure correct extension
    if os.path.exists(image_path):
        for caption in captions:
            dataset_list.append({"image_path": image_path, "text": caption})

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(dataset_list)

# Step 4: Split Dataset into Train/Test (90% Train, 10% Test)
dataset = dataset.train_test_split(test_size=0.1)

# Step 5: Convert to DatasetDict
dataset_dict = DatasetDict({"train": dataset["train"], "test": dataset["test"]})

# Step 6: Save Dataset for Future Use
dataset_dict.save_to_disk("my_dataset")

# Check Output
print(dataset_dict)

Saving the dataset (0/1 shards):   0%|          | 0/1480 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/165 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image_path', 'text'],
        num_rows: 1480
    })
    test: Dataset({
        features: ['image_path', 'text'],
        num_rows: 165
    })
})


In [6]:
dataset_dict.save_to_disk(os.path.join(INPUT_DIR, 'Microsoft-GIT'))

Saving the dataset (0/1 shards):   0%|          | 0/1480 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/165 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoProcessor, AutoModelForCausalLM

checkpoint = "microsoft/git-base"
processor = AutoProcessor.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [8]:
def transforms(example_batch):
    images = [Image.open(x).convert("RGB") for x in example_batch["image_path"]]  # Change "image" to "image_path"
    captions = [x for x in example_batch["text"]]
    inputs = processor(images=images, text=captions, padding="max_length", return_tensors="pt")
    inputs.update({"labels": inputs["input_ids"]})
    return inputs

dataset_dict["train"].set_transform(transforms)
dataset_dict["test"].set_transform(transforms)


In [39]:
import torch
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 3            |        cudaMalloc retries: 74        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   3529 MiB |  14140 MiB |  11595 GiB |  11592 GiB |
|       from large pool |   3385 MiB |  14117 MiB |  10496 GiB |  10493 GiB |
|       from small pool |    144 MiB |    144 MiB |   1099 GiB |   1098 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   3529 MiB |  14140 MiB |  11595 GiB |  11592 GiB |
|       from large pool |   3385 MiB |  14117 MiB |  10496 GiB |

In [46]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="git-image-captioning",
    learning_rate=5e-5,
    num_train_epochs=10,  # Reduce for smaller datasets
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    logging_steps=50,
    save_total_limit=2,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    push_to_hub=False,  # Set to True if pushing to Hugging Face Hub
    fp16=False,
    bf16=True,  # More stable than fp16 on some GPUs
    dataloader_num_workers=0,
    max_grad_norm=1.0,  # Prevents gradient explosion
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [48]:
from evaluate import load
import torch

wer = load("wer")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predicted = logits.argmax(-1)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions = processor.batch_decode(predicted, skip_special_tokens=True)
    wer_score = wer.compute(predictions=decoded_predictions, references=decoded_labels)
    return {"wer_score": wer_score}


In [49]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [52]:
import torch
torch.cuda.empty_cache()


In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    compute_metrics=compute_metrics,
)

trainer.train()


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.66 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.58 GiB is free. Process 8643 has 12.15 GiB memory in use. Of the allocated memory 11.22 GiB is allocated by PyTorch, and 818.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from PIL import Image
import requests

# Load test image
image_path = "images/sample.jpg"  # Change this to an actual image from your dataset
image = Image.open(image_path).convert("RGB")

device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = processor(images=image, return_tensors="pt").to(device)

# Generate Caption
generated_ids = model.generate(pixel_values=inputs.pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Generated Caption:", generated_caption)
